--- 
Project for the course in Microeconometrics | Summer 2021, M.Sc. Economics, Bonn University | [Philipp Schreiber](https://github.com/pcschreiber1)

# Replication of Henderson, Storeygard, Deichmann (2017) :
## Has climate change driven urbanization in Africa? <a class="tocSkip">   
    
    
---

<div class="alert alert-block alert-info">
    <h5><b>Downloading and viewing this notebook:</b></h5> <p> The original paper, as well as the data and code provided by the authors can be accessed <a href="https://doi.org/10.1016/j.jdeveco.2016.09.001">here</a> </p></div>

## Content:

In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker
import statsmodels.formula.api as smf
import seaborn as sns

#For spatial analysis
import geopandas as gpd
import shapely.geometry as geom
import libpysal as lp #For spatial weights

from pysal.viz import splot #exploratory analysis
from pysal.explore import esda #exploratory analysis
from pysal.model import spreg #For spatial regression

pd.options.display.float_format = "{:,.2f}".format

ModuleNotFoundError: No module named 'geopandas'

In [ ]:
from auxiliary.data_import import *
from auxiliary.plots import *
from auxiliary.simulations import *
from auxiliary.tables import *

---
# 1. Introduction 
---

In this paper Henderson et al. investigate the impact of increasing aridity on urbanization in Sub-Saharan Africa. In particular, the authors address the questions of i) *whether adverse changes in climate induce a push from rural to urban areas* and ii) *how this within-country migration affects incomes in cities*.  They attribute previous studies' mixed findings to aggregation effects resulting from using national level data and, therefore, use variation at the district and city level. To enable such an analysis the authors constructed a panel data set combining district level census data, fine resolution climate data and **night time lights (NTLs)** between 1960 and 2008. The authors' central finding is that climate shocks only produce rural-urban movements if cities are can offer alternative employment options. 

The analysis is framed by a simple model of rural-urban migration where the migration decision depends on potential wages in the rural and urban sectors, respectively. It follows that rising aridity (i.e., a decline in moisture) only draws citizens into the urban sector in the presence of a manufacturing base creating potentially tradeable goods. Otherwise, the urban sector only services the rural sector and there is no competition for labour. The study employs country-level **two-way fixed effects (2FE) regression** to generate reduced form estimates of the net effect of moisture on urbanization growth and city incomes. Importantly, the authors also provide two robustness checks for the underlying causal mechanism (i.e., aridity driving incomes): i) drawing on the Social Conflict Analysis Database, they do not find evidence of conflict driving the results and ii) at the city level, the effect of conflict on incomes is ambiguous, which does not match the data observed. In the face of the dire climate change projections we are confronted with today, the policy insight from this paper is simple: successful spatial and structural adaptation to climate change will critically depend on the capacities of cities. 

In this notebook, I replicate the core results presented in the paper by Henderson et al. (2017). In addition, I enrich the author’s analysis by a detailed discussion of the underlying identification strategy under the counterfactual framework. Importantly, I show that the Stable Unit Treatment Value Assumption (SUTVA) is violated through spatial interdependence and, in turn, **introduce a spatially explicit identification and estimate its results**. My analysis offers general support for the findings of Henderson et al. and provide an exploratory use case of a spatially explicit counterfactual model.

The remaining notebook is then structured as follows. In the next section, I discuss the literature on climate induced migration, emphasising the interdisciplinarity of the subject (Section 2). Section 3 then introduces the data set as well as the use of NTLs. The analysis is then structured around the two identification strategies. In Section 4, I first discuss the 2FE identification strategy used by the authors and then proceed to reproduce the core results. The final part of Section 4 discusses the robustness checks of the underlying causal mechanism. Section 5 develops the spatially explicit analysis. After explaining the tension between SUTVA and ignorability I use the results from a simulation study to illustrate the feasible identification strategies. I then examine the spatial dependence present in the data and estimate a set of spatial models. Section 6 offers concluding remarks. 

<span style="color:orange">**NOTE**:</span> While the structure of the discussion deviates from that of the original paper, all figures and tables with replication results follow the original numbering.


---
# 2. Theoretical Background
---

The question of the climate change on people's decision where to live has drawn attention from a variety of disciplines and academic traditions. Naturally, there is not a unique answer to such a complex socio-politico-economic-ecological phenomenon as migration. Jónsson (2010)identifies two general conceptual approaches to studying climate-migration patterns: Push-Pull factor and Multi-level Contextual driver approaches. **While Henderson et al. (2017) explicitly position themselves in the former tradition we will see below that their disaggregated analysis can be seen to bridge the gap between the two traditions**.

As the authors point out, the present study is most closely tied to the 2006 paper by Barrios et al., who use change in raifall to estimate the effect of adverse climate change on urbanization. Motivated by the concept of climate refugess, the authors posit that a generally low degree of arrigation makes the Sub-Saharan agricultural sector particularly vulnerable to rainfall shocks. Similarly, Brückner (2012) uses rainfall as an instrument for agricultural GDP and also find that a decrease in rainfall increases urbanization growth. Both studies also ascribe themselves to the Push-Pull factor framework. In contrast, the literature on Multi-level contextual drivers - mostly developed by anthropoligists and political scientists - sees the connection between climate change and  migration as a lot less clear. In particular, they emphasise the multiplicity of strategies citizens have at hand when facing climate shocks. **What is more, as Morrissey (2009) points out, it could be argued that climate change in the Sahel region because it undermines migration**, due to lack of resources. The feasibility and attractivness of migration then also critically depends on the presence of social networks. An alternative line of investigation points towards the political ecology - that is, the subjectivity of interpretation of climate change - which threatens both the externality and usefulness of ecological climate measures.

The sub-national analysis of Henderson et al. (2017) enables a significantly more nuanced perspective of migration. The emphasis on labour market arbitrage is an important step in better reflecting parts of the complexity that individuals face.

<h3><u>2.2 A labour market model of rural-urban migration</u></h3>

The authors model districts as small open economies which are differentiated by the presence of manufacturing facilities for potentially tradeable goods.  In this context, a decline in moisture should only induce local migration in industrialized districts, since in exclusievly rural districts cities are only provide agriculture with services that not traded across districts, the local agricultural sector is not competing for labor.

More precisely, the economies face fixed prices of imports and exports and are divided into a rural and an urban sector. Both produce services but the latter also may or may not produce tradeable goods. In line with standard urban models, workers living in the city face diseconomies of scale due to commuting costs to generate a stable equilibrium. Migration arbitrage etween the urban and rural sectors equalizes incomes and there is full employment. An adverse climate shock then decreases wages in the rural. Service incomes in the urban sector would also decline, however, since urban wages equalize, the wage effect is ambigous: if there is no or only a neglible manufacturing sector, city incomes also decrease and there is no migration arbitrage. But **if there is a significant manufacturing sector, the urban wages will decrease less, generating the opporunity for migration arbitrage**. In the real world, the presence of industry is endogenous of course. This issue is addressed again in the identification section. The authors fully describe the model in Appendix B, for further information on urban economic models, refer to Duranton and Puga (2004) and Desmet and Henderson (2015).

---
# 3. Data
---

I now proceed to discuss the measures of urbanization, income, climate data and industrialization that the authors use. While high-quality climate data is readily available at all levels of resolution, the same is not true for demographic and economic data. Still, if available, the large increase in the number of observations can be a strong benefit to the analysis. At the district level, the data set contains census data between 1960 to 2008 and includes 369 districts or 29 countries. The city level analysis spans from 1992 to 2008 and includes 1158 cities and towns in 42 countries.
<h3><u>3.1 Urbanization</u></h3>

Most studies employ **population data** from infrequent national census reports and rely heavily on imputations and interpolations. To avoid this, the study includes countries with at least two available censuses with the relevant information for a complete or nearly complete set of sub-national units, where either district boundaries changed little or common units over time can be defined. The authors digitized data from hardcopy census publications. Of the 32 available countries, Namibia, Congo-Brazzaville and Liberia were dropped due to issues of district definition or census frequency. The 29 remaining countries count between 2 to 5 censuses between 1960 and 2019. For estimation purposes Kenya is treated as two countries before and after the urban redefinition of the 1990s. While extensive, the sample omits several Sub-Saharan countries, most notably Nigeria, the most populous country on the continent. These countries were omitted either because the censuses lacked the required information or printed volumes were unavailable. South Africa was excluded because of Post-Apartheid migration. The district panel is unbalanced and the authors calculate the annualised urban growth as the difference in urban population averaged by length of the intercensal period. Districts vary a lot in size, a topic which is analysed more closely in section 5.

In contrast, high quality **climate data** is much more readily available at the district level. To reflect climatic agricultural potential the authors use precipitation over potential evapotranspiration (PET) (i.e. system supply over system demand) as a moisture index. The precipitation and temperature data is available from the University of Delaware gridded climate data set while PET is estimated using the Thornthwaite method. The district level climate indicator is then the average grid cell value that overlaps with the corresponding sub-national unit where cells that cross the boundary are weighted.

Sub-national **data on industrialization** is scarce in many census reports - before 1985 this is even true for national level indicators. Henderson et al. draw on the *Oxford Regional Economic Atlas, Africa* (Ady, 1965) which maps all industries by type and city location, based on an in-depth analysis from a variety of sources from the late 1950s and early 1960s. From the total 26 industries the authors exclude 10 agricultural processing industries (e.g. brewing, milling) and use the count of different industries as the industrialization indicator. Comparing this index to 2000 IPSUM data on manufacturing, the 1960 industry count still functions as a good proxy. The full 26 industries are used in most robustness checks. Since the value is not updated, it can be interpreted as the likelihood of having industry at time $t$.

In [1]:
map_data_section()

NameError: name 'map_data_section' is not defined